In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.table as table
from astropy.table import Table
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.mast import Catalogs
from astropy.coordinates import SkyCoord
from tqdm import tqdm

In [42]:
# table_B = Table.read('table_B.fits', format='fits')

In [2]:
# table_0 = Table.read('table_cut.fits', format='fits')
table_0 = Table.read('table_cut.fits', format='fits')
members = Table.read('./data/hunt_clusters/members.csv', format='csv')
clusters = Table.read('./data/hunt_clusters/clusters.csv', format='csv')
# uv_excess = [7,9,13,20,21,23,28]
# no_excess = [8,10,12,16,24]
# no_galex = [0,1,2,3,4,5,6,11,14,15,17,18,19,22,25,26,27]

In [ ]:
## query gaia for all sources in table_0- get relevant columns

query1 = '''SELECT source_id, ra , dec, ra_error ,dec_error,parallax,parallax_error, astrometric_excess_noise, astrometric_excess_noise_sig,
 astrometric_params_solved, pseudocolour, visibility_periods_used, astrometric_sigma5d_max, ruwe, phot_g_mean_flux, phot_g_mean_flux_error, phot_g_mean_mag,
   phot_g_mean_flux_over_error, phot_bp_mean_flux, phot_bp_mean_flux_error, phot_bp_mean_flux_over_error, phot_bp_mean_mag, phot_rp_mean_flux,
     phot_rp_mean_flux_error, phot_rp_mean_flux_over_error, phot_rp_mean_mag, bp_rp, bp_g, g_rp, radial_velocity, radial_velocity_error, l, b,
       ecl_lon, ecl_lat, has_xp_continuous, has_xp_sampled, has_rvs, has_epoch_photometry, has_epoch_rv
    FROM gaiadr3.gaia_source
    WHERE source_id IN {id_lst}
    '''.format(id_lst = tuple(table_0['source_id'].data))
job1 = Gaia.launch_job(query= query1)
table_1 = job1.get_results()

## query nss for all soruces in table_0 - get relevant columns

query2 = '''SELECT source_id, parallax, parallax_error , pmra, pmra_error,
 pmdec, pmdec_error, a_thiele_innes, a_thiele_innes_error, b_thiele_innes, b_thiele_innes_error, f_thiele_innes, f_thiele_innes_error,
  g_thiele_innes, g_thiele_innes_error, c_thiele_innes, c_thiele_innes_error, h_thiele_innes, h_thiele_innes_error, period, period_error,
   t_periastron, t_periastron_error, eccentricity, eccentricity_error, center_of_mass_velocity, center_of_mass_velocity_error,
    semi_amplitude_primary, semi_amplitude_primary_error, semi_amplitude_secondary, semi_amplitude_secondary_error, mass_ratio,
     mass_ratio_error, fill_factor_primary, fill_factor_primary_error, fill_factor_secondary, fill_factor_secondary_error, inclination,
      inclination_error, arg_periastron, arg_periastron_error, temperature_ratio, temperature_ratio_error, temperature_ratio_definition, 
      bit_index, corr_vec, goodness_of_fit, efficiency, significance, flags, g_luminosity_ratio, astrometric_jitter, nss_solution_type
    FROM gaiadr3.nss_two_body_orbit
    WHERE source_id IN {id_lst}
    '''.format(id_lst = tuple(table_0['source_id'].data))
job2 = Gaia.launch_job(query= query2)
table_2 = job2.get_results()

## join tables, reorder columns by importance

table_0 = table.join(table_2,table_1,keys=['source_id'],join_type='left')

## take parallax from NSS if available, otherwise take from Gaia source
par_col = [pgaia if np.ma.is_masked(pnss) else pnss for pnss,pgaia in zip(table_0['parallax_1'],table_0['parallax_2'])]
par_err = [egaia if np.ma.is_masked(enss) else enss for enss,egaia in zip(table_0['parallax_error_1'],table_0['parallax_error_2'])]
par_col = Table.Column(par_col, name='parallax',unit='mas')
par_err = Table.Column(par_err, name='parallax_error',unit='mas')
table_0.add_columns([par_col,par_err])
table_0.remove_columns(['parallax_1','parallax_2','parallax_error_1','parallax_error_2'])

table_0['parallax_over_error'] = table_0['parallax']/table_0['parallax_error']
table_0['mg'] = table_0["phot_g_mean_mag"] + 5 * np.log10(table_0["parallax"]) - 10
cols = table_0.colnames
first_cols = ['source_id','ra','dec','parallax','phot_g_mean_mag','bp_rp']
for c in first_cols:
    cols.remove(c)
cols = first_cols + cols
table_0 = table_0[cols]

In [8]:
## convert metallicity catalog h5 files to fits

import h5py 
from astropy.table import Table
import os
from glob import glob

# Input location
# catalog_h5_list = glob('c:/Users/ASUS/Dropbox (Weizmann Institute)/AMRF IFMR/data/metallicity/stellar_params_catalog_*.h5')
catalog_h5_list = glob('/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/data/metallicity/stellar_params_catalog_*.h5')
catalog_h5_list.sort()

for catalog_h5_loc in catalog_h5_list:
    print(f"Loading {catalog_h5_loc}")
    output_fits = Table()
    with h5py.File(catalog_h5_loc, 'r') as f:
        for i, key in enumerate(f.keys()):
            print(f"Loading {i+1}/{len(f.keys())}: {key}")
            output_fits[key] = f[key][:]
    base_fn,_ = os.path.splitext(catalog_h5_loc)
    catalog_fits_loc = base_fn + '.fits'
    print(f"Saving to {catalog_fits_loc}")
    output_fits.write(catalog_fits_loc)

In [ ]:
## read first metallicity catalog file, create metallicity column, remove other columns

# ColDefs(name = 'chi2_opt'; format = 'E'
#     name = 'dec'; format = 'E'
#     name = 'feh_confidence'; format = 'E'
#     name = 'gdr3_source_id'; format = 'K'
#     name = 'ln_prior'; format = 'E'
#     name = 'logg_confidence'; format = 'E'
#     name = 'quality_flags'; format = 'B'
#     name = 'ra'; format = 'E'
#     name = 'stellar_params_err'; format = '5E'; dim = '(5)'
#     name = 'stellar_params_est'; format = '5E'; dim = '(5)'
#     name = 'teff_confidence'; format = 'E') 


hdul_7 = fits.open('c:/Users/ASUS/Dropbox (Weizmann Institute)/AMRF IFMR/data/catalogs/metallicity/stellar_params_catalog_00.fits')
coldef = fits.ColDefs([hdul_7[1].columns[3],hdul_7[1].columns[8],hdul_7[1].columns[9],hdul_7[1].columns[6]])
table_7 = Table(hdul_7[1].from_columns(coldef).data)
table_7.add_column(table_7['stellar_params_est'][:,1]*u.dex,name='Fe_H_est')
table_7.add_column(table_7['stellar_params_err'][:,1]*u.dex,name='Fe_H_error')
table_7.remove_columns(['stellar_params_err','stellar_params_est'])
table_7.rename_column('gdr3_source_id','source_id')
hdul_7.close()

## read all metallicity catalog files, create metallicity column, remove other columns, append to table_7
for i in range(1,10):
    filename = f'c:/Users/ASUS/Dropbox (Weizmann Institute)/AMRF IFMR/data/metallicity/stellar_params_catalog_0{i}.fits'
    print(filename)
    hdul_temp = fits.open(filename)
    coldef = fits.ColDefs([hdul_temp[1].columns[3],hdul_temp[1].columns[8],hdul_temp[1].columns[9],hdul_temp[1].columns[6]])
    table_temp = Table(hdul_temp[1].from_columns(coldef).data)
    hdul_temp.close()
    table_temp.add_column(table_temp['stellar_params_est'][:,1]*u.dex,name='Fe_H_est')
    table_temp.add_column(table_temp['stellar_params_err'][:,1]*u.dex,name='Fe_H_error')
    table_temp.remove_columns(['stellar_params_err','stellar_params_est'])
    table_temp.rename_column('gdr3_source_id','source_id')
    table_7 = table.vstack([table_7,table_temp])

table_0 = table.join(table_0,table_7,keys=['source_id'],join_type='left')

In [ ]:
## get metallicity for all cluster members, take median

# hdul_7 = fits.open('c:/Users/ASUS/Dropbox (Weizmann Institute)/AMRF IFMR/old/data/catalogs/metallicity/stellar_params_catalog_00.fits')
hdul_7 = fits.open('/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/old/data/catalogs/metallicity/stellar_params_catalog_00.fits')
coldef = fits.ColDefs([hdul_7[1].columns[3],hdul_7[1].columns[8],hdul_7[1].columns[9],hdul_7[1].columns[6]])
table_7 = Table(hdul_7[1].from_columns(coldef).data)
table_7.add_column(table_7['stellar_params_est'][:,1]*u.dex,name='Fe_H_est')
table_7.add_column(table_7['stellar_params_err'][:,1]*u.dex,name='Fe_H_error')
table_7.remove_columns(['stellar_params_err','stellar_params_est'])
table_7.rename_column('gdr3_source_id','source_id')
hdul_7.close()

## read all metallicity catalog files, create metallicity column, remove other columns, append to table_7
for i in range(1,10):
    # filename = f'c:/Users/ASUS/Dropbox (Weizmann Institute)/AMRF IFMR/old/data/catalogs/metallicity/stellar_params_catalog_0{i}.fits'
    filename = f'/home/oreni/Dropbox (Weizmann Institute)/AMRF IFMR/old/data/catalogs/metallicity/stellar_params_catalog_0{i}.fits'
    print(filename)
    hdul_temp = fits.open(filename)
    coldef = fits.ColDefs([hdul_temp[1].columns[3],hdul_temp[1].columns[8],hdul_temp[1].columns[9],hdul_temp[1].columns[6]])
    table_temp = Table(hdul_temp[1].from_columns(coldef).data)
    hdul_temp.close()
    table_temp.add_column(table_temp['stellar_params_est'][:,1]*u.dex,name='Fe_H_est')
    table_temp.add_column(table_temp['stellar_params_err'][:,1]*u.dex,name='Fe_H_error')
    table_temp.remove_columns(['stellar_params_err','stellar_params_est'])
    table_temp.rename_column('gdr3_source_id','source_id')
    table_7 = table.vstack([table_7,table_temp])

table_7 = table_7[table_7['quality_flags'] < 8]
table_0['Fe_H_cluster'] = np.full(len(table_0),np.nan)
table_0['Fe_H_cluster_std'] = np.full(len(table_0),np.nan)

for i,id in tqdm(enumerate(table_0['id']),total=len(table_0['id'])):
    nbr = members[members['id']==id]
    fh_arr = table_7[np.isin(table_7['source_id'],nbr['source_id'])]['Fe_H_est']
    table_0[i]['Fe_H_cluster'] = np.median(fh_arr)
    table_0[i]['Fe_H_cluster_std'] = np.std(fh_arr)

In [40]:
## Get cluster data for each source
members = Table.from_pandas(pd.read_csv('./data/hunt_clusters/members.csv',usecols=['source_id','id','probability']))
clusters = Table.from_pandas(pd.read_csv('./data/hunt_clusters/clusters.csv',usecols=['id','log_age_16','log_age_50','log_age_84','a_v_16','a_v_50','a_v_84','n_stars']))

table_0 = table.join(table_0, members, keys='source_id', join_type='left')
table_0 = table.join(table_0, clusters, keys='id', join_type='left')

In [49]:
## Astroquery GALEX data
# i = 40
# ra = table_0[i]['ra']
# dec = table_0[i]['dec']
# r_arcsec = 10
# r_deg = r_arcsec/3600
# catalog_data = Catalogs.query_object(f'{ra} {dec}',catalog='Galex',radius = r_deg)

# print(np.min(catalog_data['distance_arcmin'].data)*60)


for c in ['distance_arcmin','nuv_mag','nuv_magerr','fuv_mag','fuv_magerr']:
    table_0[c] = np.full(len(table_0),np.nan)


for i in tqdm(range(len(table_0))):
    ra = table_0[i]['ra']
    dec = table_0[i]['dec']
    r_arcsec = 2
    r_deg = r_arcsec/3600
    catalog_data = Catalogs.query_object(f'{ra} {dec}',catalog='Galex',radius = r_deg)

    if len(catalog_data)>0:
        for c in ['distance_arcmin','nuv_mag','nuv_magerr','fuv_mag','fuv_magerr']:
            table_0[c][i] = catalog_data[c][0]

100%|██████████| 1677/1677 [26:45<00:00,  1.04it/s]


In [51]:
## deal with problematic data types

col = table.Column([str(cvec) for cvec in table_0['corr_vec']],name = 'corr_vec',dtype = str)
table_0['nss_solution_type'] = table_0['nss_solution_type'].astype(str)
table_0['corr_vec'] = col

In [27]:
## sort columns by importance

cols = table_0.colnames
first_cols = ['source_id','ra','dec','parallax','mg','phot_g_mean_mag','bp_rp','log_age_50','av_bayestar','av_acrist','Fe_H_est','probability','nuv_mag']
for c in first_cols:
    cols.remove(c)
cols = first_cols + cols
table_0 = table_0[cols]

In [ ]:
## query extinction (northern)

# from dustmaps.config import config
# config['data_dir'] = '/path/to/store/maps/in'

# import dustmaps.bayestar
# dustmaps.bayestar.fetch()

# from dustmaps.bayestar import BayestarQuery

# bayestar = BayestarQuery(max_samples=2, version='bayestar2015')
# l = table_0['l']
# b = table_0['b']
# d = np.abs(1000/table_0['parallax'])
# coord = SkyCoord(l=l*u.deg, b=b*u.deg, distance=d*u.pc, frame='galactic')
# ebv = 0.884 * bayestar(coord, mode='median')

In [29]:
## query extinction (southern)
# from urllib import request
# av = []
# with tqdm(total=len(table_0)) as pbar:
#     for ra,dec,par in zip(table_0['ra'],table_0['dec'],table_0['parallax']):
#         d = np.abs(1000/par)
#         with  request.urlopen(f'https://astro.acri-st.fr/gaia_dev/extinction?frame=icrs&vlong={ra}&ulong=deg&vlat={dec}&ulat=deg&distance={d}') as response:
#             html = response.read()
#             av.append(float(html.split(b'\n')[1].split(b',')[1]))
#         pbar.update(1)

  0%|          | 0/1677 [00:00<?, ?it/s]

100%|██████████| 1677/1677 [10:48<00:00,  2.58it/s]


In [29]:
## save table
table_0.write('table.fits', format='fits',overwrite=True)

In [41]:
## cut table_B from 660 to relevant non class I sources

# table_0 = Table.read('table_B.fits', format='fits')

# cond = (table_0['probability'].data>=0.99) & (table_0['m1'].data/table_0['m1_err'].data>=10) &  (table_0['classI_prob'].data <= 0.1)

# table_1 = table_0[cond]

# table_1.write('table_cut.fits', format='fits',overwrite=True)